In [2]:
import numpy as np
import cv2 
import pandas as pd 
import os 
import pickle

In [3]:
#Face detection model
model = 'C:/Users/ASUS/Project/Data/res10_300x300_ssd_iter_140000_fp16.caffemodel'

# mo ta kien truc 
faceDetectionProto = 'C:/Users/ASUS/Project/Data/deploy.prototxt'
# mo hinh mo ta khuon mat 
faceDescriptorModel = 'C:/Users/ASUS/Project/Data/openface.nn4.small2.v1.t7'
# Su dung opencv Dnn dc mo hinh nhan dien khuon mat 
detectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto,model) # can hoc readNet from 
# doc mo hinh mo ta khuon mat tu Torch 
descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptorModel)
detectorModel

< cv2.dnn.Net 00000280985F5650>

In [4]:
# danh sach hinh anh chuan bi 
faces = './Pillow_01/Faces'
os.listdir(faces)

['.ipynb_checkpoints', 'Congphuong', 'Kante', 'Messi', 'obama', 'Ronaldo']

In [5]:
img1 = cv2.imread('./Pillow_01/img/th.jpg')
cv2.imshow('Image',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
img = img1.copy()
# heigh and width cua anh
h,w = img.shape[:2]
# prepare input data for face detection
blob = cv2.dnn.blobFromImage(img,1.0,(300,300),(104,177,123),swapRB = False,crop = False)
# thiet lap dau vao cho mo hinh 
detectorModel.setInput(blob)
detection = detectorModel.forward()
# kiem tra co khuon mat k 
if len(detection) > 0:
    i = np.argmax(detection[0,0,:,2])
    #chon khuon mat co confidence cao nhat
    confidence = detection[0,0,i,2]
    if confidence > 0.5:
        # tinh hop bao quanh khuon mat
        box = detection[0,0,i,3:7] * np.array([w,h,w,h])
        (start_x,start_y,end_x,end_y) = box.astype('int')
    # trich xuat khuon mat ra
    roi = img[start_y: end_y,start_x:end_x]
    
    # chuan bi dau vao cho model 
    faceBlob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB =True,crop = True)
    # thiet lap bien dau vao cho mo hinh
    descriptorModel.setInput(faceBlob)
    # trich xuat dac trung 
    vectors = descriptorModel.forward()
    print(vectors)

[[-0.00600992 -0.10820479 -0.15893662  0.03501324  0.07737229  0.09475695
   0.06557967 -0.03499735 -0.06985712 -0.01200356 -0.03268898  0.11469947
   0.14573479  0.01278464 -0.11763225 -0.02252352 -0.04224468  0.05441085
  -0.01377466  0.17020065  0.1549087  -0.2518361   0.03465542  0.14802967
  -0.00244369 -0.0114529  -0.11137871 -0.25309733 -0.05731896  0.12407583
  -0.00208241  0.07495363 -0.04824748 -0.0026366  -0.0356188   0.07201789
  -0.12991472  0.04794511 -0.12320963 -0.13383079  0.01461774 -0.03282278
   0.00592746  0.11673947 -0.07860111 -0.04042094  0.1544471  -0.0016945
  -0.06642781 -0.02930678 -0.03971313 -0.21764773  0.05258583 -0.04146545
   0.14580679  0.07114135 -0.05799212  0.17709923  0.07473099 -0.1235932
   0.03006213  0.07693341  0.14984432 -0.08798128  0.10588823 -0.01391415
  -0.04001203  0.00667599 -0.05066695 -0.0548952  -0.0345403   0.08663426
  -0.04807851 -0.10602875 -0.04761557  0.09967194  0.01685676  0.05069891
   0.0487206   0.02118032 -0.11860822 -0

In [7]:
# 1 function cho nhieu anh 
def detect(path):
    img1 = cv2.imread(path)
    img = img1.copy()
    # heigh and width cua anh
    h,w = img.shape[:2]
    # prepare input data for face detection
    blob = cv2.dnn.blobFromImage(img,1.0,(300,300),(104,177,123),swapRB = False,crop = False)
    # thiet lap dau vao cho mo hinh 
    detectorModel.setInput(blob)
    detection = detectorModel.forward()
    # kiem tra co khuon mat k 
    if len(detection) > 0:
        #chon khuon mat co confidence cao nhat
        i = np.argmax(detection[0,0,:,2])
        
        confidence = detection[0,0,i,2]
        if confidence > 0.5:
            # tinh hop bao quanh khuon mat
            box = detection[0,0,i,3:7] * np.array([w,h,w,h])
            (start_x,start_y,end_x,end_y) = box.astype('int')
        # trich xuat khuon mat ra
        roi = img[start_y: end_y,start_x:end_x]
        
        # chuan bi dau vao cho model 
        faceBlob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB =True,crop = True)
        # thiet lap bien dau vao cho mo hinh
        descriptorModel.setInput(faceBlob)
        # trich xuat dac trung 
        vectors = descriptorModel.forward()
        return vectors 

In [8]:
#apply all image 
# khoi tao dictionary
data = dict(data= [],label = [])
mydir = './Pillow_01/Faces'
folders = os.listdir(mydir)
# duyet qua tung thu muc 
for folder in folders:
    path = mydir + '/' +folder 
    files = os.listdir(path)
    for file in files:
        try:
            # Gọi hàm trích xuất đặc trưng
            vector = detect(path + '/'+file)
            if vector is not None:
            #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
                data['data'].append(vector)
                data['label'].append(folder)
        except:
            pass

In [9]:
data.keys()

dict_keys(['data', 'label'])

In [10]:
# Tạo một series từ dictionary
labelSeries = pd.Series(data['label'])
# Đếm số lần xuất hiện của mỗi nhãn
labelCounts = labelSeries.value_counts()
print(labelCounts)

Congphuong    2
Kante         2
Messi         2
obama         2
Ronaldo       2
Name: count, dtype: int64


In [11]:

# Save data -> file

fileName = './Pillow_01/img/Results.pickle'
pickle.dump(data, open(fileName, mode='wb'))